# At First Scraping The Data

In [ ]:
import requests
from bs4 import BeautifulSoup
import re
import time
import random
from concurrent.futures import ThreadPoolExecutor, as_completed
from collections import defaultdict
import pandas as pd

category_name_list = []
all_phones_pages_link_list = []
all_phones_link_list = []
all_headphones_pages_link_list = []
all_headphones_link_list = []
all_covers_pages_link_list = []
all_covers_link_list = []
all_screens_pages_link_list = []
all_screens_link_list = []
all_power_banks_pages_link_list = []
all_power_banks_link_list = []
all_chargers_pages_link_list = []
all_chargers_link_list = []
headers = {
"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36",
"Accept-Encoding": "gzip, deflate",
"Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,/;q=0.8",
"DNT": "1",
"Connection": "close",
"Upgrade-Insecure-Requests": "1",
}
def scrape_proxies(url, limit=101):
    try:
        # Fetch the webpage content
        response = requests.get(url)
        response.raise_for_status()  # Raise an exception for bad status codes
        
        # Parse the HTML content
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Find all table rows
        rows = soup.find_all('tr')
        
        # Extract IP and Port from each row and format as requested
        proxies = [
            {'http': f"{row.find_all('td')[0].text}:{row.find_all('td')[1].text}"}
            for row in rows[:limit]
            if len(row.find_all('td')) >= 2
        ]
        
        return proxies
    
    except requests.RequestException as e:
        # print(f"An error occurred while fetching the webpage: {e}")
        return []
    except Exception as e:
        # print(f"An unexpected error occurred: {e}")
        return []

mohamed = "https://free-proxy-list.net"  # Replace with the actual mohamed
proxies = scrape_proxies(mohamed)

for i in range(1,34):
    product_link = f'https://www.amazon.nl/s?i=electronics&rh=n%3A16366059031&s=popularity-rank&fs=true&page={i}&language=en_GB'
    all_phones_pages_link_list.append(product_link)
proxy_failures = defaultdict(int)
MAX_FAILURES = 101
def get_proxy():
    working_proxies = [p for p, failures in proxy_failures.items() if failures < MAX_FAILURES]
    return random.choice(working_proxies) if working_proxies else None


def get_text_or_none(tag, attribute_name=None):
    if tag:
        if attribute_name:
            return tag.get(attribute_name, None)
        return tag.get_text(strip=True)
    return None
def make_request(url, max_retries=51):        
    for attempt in range(max_retries):
        proxy = get_proxy()
        if not proxy:
            # print("No working proxies available. Trying without proxy.")
            try:
                response = requests.get(url, headers=headers)
                response.raise_for_status()
                return response
            except requests.RequestException as e:
                # print(f"Attempt {attempt + 1} failed without proxy: {e}")
                time.sleep(random.uniform(1, 2))
                continue

        try:
            response = requests.get(url, headers=headers, proxies={'http': proxy})
            response.raise_for_status()
            return response
        except requests.RequestException as e:
            # print(f"Attempt {attempt + 1} failed with proxy {proxy}: {e}")
            proxy_failures[proxy] += 1
            if proxy_failures[proxy] >= MAX_FAILURES:
                # print(f"Proxy {proxy} has failed {MAX_FAILURES} times and will no longer be used.")
                time.sleep(random.uniform(1, 2))
    
    # raise Exception(f"Failed to retrieve {url} after {max_retries} attempts")

def get_all_phones_link(link, max_retries=35):
    retry_count = 0
    
    while retry_count < max_retries:
        try:
            page = make_request(link)
            soup = BeautifulSoup(page.content, "html.parser")
            all_phones = soup.find_all("div", class_="a-section a-spacing-none a-spacing-top-small s-title-instructions-style")
            
            if all_phones:
                for phone in all_phones:
                    phone_link = phone.find_all("a")[0].get("href")
                    phone_link = f"https://www.amazon.nl{phone_link}"
                    all_phones_link_list.append(phone_link)
                
                print(f"Found {len(all_phones)} phones on page {link}")
                return
            
            else:
                retry_count += 1
                # print(f"No phones found on page {link}, retrying... ({retry_count}/{max_retries})")
                time.sleep(random.uniform(1, 2))  # Random delay before retrying

        except Exception as e:
            # print(f"Error getting phone links from {link}: {e}")
            retry_count += 1
            time.sleep(random.uniform(1, 2))  # Random delay before retrying
    
    # print(f"Failed to find phones on {link} after {max_retries} retries")


# Modified scrape_product_data function
def scrape_product_data(link, max_retries=500):
    for attempt in range(max_retries):
        # time.sleep(random.uniform(1, 2))
        try:
            page = make_request(link)
            soup = BeautifulSoup(page.content, "html.parser")

        
            camera_info = None
            for li in soup.find_all('li', class_='a-spacing-mini'):
                if 'MP' in li.text:
                    camera_info = re.search(r'(\d+MP)', li.text)
                    if camera_info:
                        camera_info = camera_info.group(1)
                        break
    
            color_row = soup.find('tr', class_='a-spacing-small po-color')
            color = None  # Initialize color
            if color_row:
                # If the color row exists, find the span inside it and get the text
                color = get_text_or_none(color_row.find('span', class_='a-size-base po-break-word'))
            
            elif soup.find('table', id ='productDetails_techSpec_section_1'):
                color_row2 = soup.find('th', string='Color')
                if color_row2:
                    color = color_row2.find_next('td').get_text(strip=True)
            
            else:
                # Check for color in div with id='variation_color_name'
                outerSelect = soup.find('div', id='variation_color_name')
                if outerSelect:
                    selection_row = outerSelect.find('span', class_='selection')
                    if selection_row:
                        color = get_text_or_none(selection_row)
    
            price = get_text_or_none(soup.find('div', class_='a-section a-spacing-micro').find('span', class_='a-offscreen')
                        if soup.find('div', class_='a-section a-spacing-micro') else None)
            if not price:
                price = get_text_or_none(soup.find('span', class_='a-price-whole'))
    
            Website = 'Amazon NL'
            Currency = 'EUR'
    
            # Extract other product data
            data = {
                # "category": category_name_list[0] if category_name_list else "unknown-category",
                "Title": get_text_or_none(soup.find('span', id='productTitle')),
                "Brand": get_text_or_none(soup.find('tr', class_='a-spacing-small po-brand').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-brand') else None),
                "OS": get_text_or_none(soup.find('tr', class_='a-spacing-small po-operating_system').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-operating_system') else None),
                "RAM": get_text_or_none(soup.find('tr', class_='a-spacing-small po-ram_memory.installed_size').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-ram_memory.installed_size') else None),
                "CPU": get_text_or_none(soup.find('tr', class_='a-spacing-small po-cpu_model.family').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-cpu_model.family') else None),
                "Storage": get_text_or_none(soup.find('tr', class_='a-spacing-small po-memory_storage_capacity').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-memory_storage_capacity') else None),
                "Screen Size": get_text_or_none(soup.find('tr', class_='a-spacing-small po-display.size').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-display.size') else None),
                "Resolution": get_text_or_none(soup.find('tr', class_='a-spacing-small po-resolution').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-resolution') else None),
                "CPU Speed": get_text_or_none(soup.find('tr', class_='a-spacing-small po-cpu_model.speed').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-cpu_model.speed') else None),
                "Model": get_text_or_none(soup.find('tr', class_='a-spacing-small po-model_name').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-model_name') else None),
                "Wireless Provider": get_text_or_none(soup.find('tr', class_='a-spacing-small po-wireless_provider').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-wireless_provider') else None),
                "Cellular Technology": get_text_or_none(soup.find('tr', class_='a-spacing-small po-cellular_technology').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-cellular_technology') else None),
                "Color": color,
                "Refresh Rate": get_text_or_none(soup.find('tr', class_='a-spacing-small po-refresh_rate').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-refresh_rate') else None),
                "SIM Count": get_text_or_none(soup.find('tr', class_='a-spacing-small po-sim_card_slot_count').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-sim_card_slot_count') else None),
                "Wireless Technology": get_text_or_none(soup.find('tr', class_='a-spacing-small po-wireless_network_technology').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-wireless_network_technology') else None),
                "Price": price,
                "Price Before Promotion": get_text_or_none(soup.find('span', class_='a-price a-text-price').find('span', class_='a-offscreen') if soup.find('span', class_='a-price a-text-price') else None),
                "Rate": get_text_or_none(soup.find('span', class_='a-icon-alt')),
                "Camera": camera_info,
                "IMG": get_text_or_none(soup.find('img', class_='a-dynamic-image'), 'data-old-hires') or get_text_or_none(soup.find('img', class_='a-dynamic-image'), 'src'),
                "Website" : Website,
                'Currancy' : Currency,
                "URL" : link
            }
    
            # Extract reviews
            reviews_from_egypt = []
            reviews_outside_egypt = []
    
            review_elements = soup.find_all("div", class_="a-section review aok-relative")
            for review in review_elements:
                reviewer_name = get_text_or_none(review.find("span", class_="a-profile-name"))
                rating = get_text_or_none(review.find("span", class_="a-icon-alt"))
                review_date = get_text_or_none(review.find("span", class_="a-size-base a-color-secondary review-date"))
                review_text = get_text_or_none(review.find("span", class_="a-size-base review-text review-text-content"))
                
                review_info = f"Name: {reviewer_name}, Rating: {rating}, Date: {review_date}, Review: {review_text}"
                
                if "Verified Purchase" in review.text:
                    reviews_from_egypt.append(review_info)
                else:
                    reviews_outside_egypt.append(review_info)
    
            data["Reviews"] = reviews_from_egypt
            if data['Title']:
                print(f"Successfully scraped data for {data['Title']}")
                return data
            else:
                continue
                # print(f"No title found for {link}, retrying... (Attempt {attempt + 1}/{max_retries})")
        except Exception as e:
            continue
            # print(f"Error scraping {link}: {e}. Retrying... (Attempt {attempt + 1}/{max_retries})")
    
    print(f"Failed to scrape {link} after {max_retries} attempts")
    return None


# Main execution
    # Get all phone links with retries on individual pages
for proxy in proxies:
    proxy_failures[proxy['http']] = 0
with ThreadPoolExecutor(max_workers=10) as executor:
    executor.map(get_all_phones_link, all_phones_pages_link_list)

print(f"Total phone links found: {len(all_phones_link_list)}")

if len(all_phones_link_list) == 0:
    print("No phone links found. Exiting...")
else:
    # Scrape product data
    results = []
    with ThreadPoolExecutor(max_workers=50) as executor:
        future_to_url = {executor.submit(scrape_product_data, url): url for url in all_phones_link_list}
        for future in as_completed(future_to_url):
            url = future_to_url[future]
            try:
                data = future.result()
                if data:
                    results.append(data)
            except Exception as exc:
                print(f'{url} generated an exception: {exc}')
    
    print(f"Successfully scraped data for {len(results)} phones")
    
    df1 = pd.DataFrame(results)
    
    if not df1.empty:
        print("Data scraping and saving completed.")
    else:
        print("No data was scraped. Check the logs for errors.")

In [ ]:
df1

# Then Cleaning The Scraped Data

In [ ]:
import logging
import ast
import json
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import re
null = None
df=df1.copy()

def scrape_the_price(url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.121 Safari/537.36'
    }
    PageToScrape = requests.get(url, headers=headers)
    if PageToScrape.status_code == 200:
        soup = BeautifulSoup(PageToScrape.text, 'html.parser')
        convert_tag = soup.find('div', class_='YMlKec fxKbKc')
        convert = convert_tag.get_text(strip = True)
        return float(convert)
SAR_USD = scrape_the_price("https://www.google.com/finance/quote/SAR-USD")
EUR_USD = scrape_the_price("https://www.google.com/finance/quote/EUR-USD")
EGP_USD = scrape_the_price("https://www.google.com/finance/quote/EGP-USD")



def convert_price_before(value):
    value =str(value)
    if isinstance(value, str):
        try:
            value = value.upper()
            value = value.replace(',', '')
            value = value.replace('SAR','')
            value = value.replace('EUR','')
            value = value.replace('€','')
            return float(value)
        except ValueError:
            return None
    else:
        return None
def convert_rate(value):
    value =str(value)
    if isinstance(value, str):
        try:
            value = value.lower()
            value = value.replace(',', '')
            value = value.replace('out of 5 stars','')
            return float(value)
        except ValueError:
            return None
    else:
        return None
def convert_to_gb(value):
    value = str(value)
    if isinstance(value, str):
        value = value.replace(',', '')
        value = value.upper()
        if 'GB' in value:
            try:
                return float(value.replace('GB', ''))
            except ValueError:
                return np.nan
        elif 'MB' in value:
            try:
                return float(value.replace('MB', '')) / 1024 
            except ValueError:
                return np.nan
        elif 'TB' in value:
            try:
                return float(value.replace('TB', '')) * 1024 
            except ValueError:
                return np.nan
        elif 'LESS THAN' in value:
            return np.nan 
        else:
            return np.nan 
    elif isinstance(value, (int, float)):
        return float(value) 
    else:
        return np.nan
def convert_screen_size(value):
    value = str(value)
    value = value.upper()
    if 'INCHES' in value:
        value = value.replace('INCHES','')
        return float(value)
    elif 'INCH' in value:
        value = value.replace('INCH','')
        return float(value)
    else:
        return None
def convert_resolution(value):
    value = str(value)
    if value and isinstance(value, str):
        try:
            width, height = value.lower().split('x')
            width = float(width)
            height = float(height)
            return width, height
        except ValueError:
            return None, None
    else:
        return None, None
def change_price_to_USD(value,convert):
    if value :
        value = value*convert
        return value
    else:
        return None
def change_to_USD(value):
    return 'USD'
def find_cpu_in_title(title):
    cpu_sizes=['Octa Core', 'Hexa Core', 'Quad Core', 'Deca Core']
    if title:
        title_lower = title.lower()
        if 'cpu' in title_lower  :
            for cpu in cpu_sizes:
                if re.search(r'\b' + re.escape(cpu.lower()) + r'\b', title_lower, re.IGNORECASE):
                    return cpu
    return None
def find_color_in_title(title):
    allcolors = [
    "Rose Gold", "Navy Blue","Light Blue","Cobalt Violet","Awesome Graphite","Titanium Black","Wave Green","Emerald Green","Titanium Silver",
    "Midnight Blue", "Forest Green", "Pink", "Purple", "Lavender","Charcoal Ink","Carbon Grey","Glamorous Green","Flowing Silver",
    "Orange", "Bronze", "Copper", "Champagne", "Beige", "Coral", "Midnight Black","Cobalt Violet",
    "Turquoise", "Teal", "Mint Green", "Burgundy", "Aqua", "Cyan", "Lime Green","Rose Red",
    "Space Gray", "Graphite", "Alpine Green", "Starlight", "Pacific Blue", "Ceramic White","Moonlight White"
    "Jet Black", "Matte Black", "Phantom Black", "Phantom Silver", "Mystic Bronze", "Awesome Navy"
    "Mystic Green", "Cosmic Gray", "Cosmic Black", "Pearl White", "Glossy White", "Awesome Lime","Silver Shadow",
    "Aura Glow", "Aurora Blue", "Prism White", "Cloud Pink", "Midnight Green", "Mint","Charcoal",
    "Sunset Gold", "Ocean Blue", "Thunder Purple", "Starry Night", "Twilight", "Clear","Navy","Chartreuse",
    "Gradient Purple", "Gradient Blue", "Gradient Red","Gradient Pink", "Marble White", "Graphite","Violet",
    "Opal White", "Crystal Blue", "Ice Blue", "Sunrise Red", "Fiery Red","Blue","Black", "White", "Gray", "Silver", "Gold", "Red","Yellow", "Green", "Brown"
]
    for color in allcolors:
        if color.lower() in title.lower():
            return color
    return None
def find_strg_in_title(title):
    storage_sizes =["1TB","512GB", "256GB", "128GB", "64GB", "32GB", "16GB", "8GB"]
    if title:
        title_lower = title.lower()
        # Check for keywords related to RAM
        
        if 'storage' in title_lower  :
            for storage in storage_sizes:
                if re.search(r'\b' + re.escape(storage.lower()) + r'\b', title_lower, re.IGNORECASE):
                    return storage
    return None
def find_ram_in_title(title):
    ram_sizes = ["1GB","2GB", "3GB","4GB","6GB","8GB","12GB","16GB"]
    if title:
        title_lower = title.lower()
        # Check for keywords related to RAM
        if 'ram' in title_lower or 'memory' in title_lower:
            for ram in ram_sizes:
                if re.search(r'\b' + re.escape(ram.lower()) + r'\b', title_lower, re.IGNORECASE):
                    return ram
    return None
df_nl=df
df_nl.columns = [col.lower() for col in df_nl.columns]
df_nl.rename(columns={'main camera': 'camera', 'network': 'cellular technology','number of sim':'sim count',}, inplace=True)
df_nl['CPU'] = df_nl.apply(lambda row: find_cpu_in_title(row['title']) or row['cpu'], axis=1)
df_nl['color'] = df_nl.apply(lambda row: find_color_in_title(row['title']) or row['color'], axis=1)
df_nl['Storage'] = df_nl.apply(lambda row: find_strg_in_title(row['title']) or row['storage'], axis=1)
df_nl['RAM'] = df_nl.apply(lambda row: find_ram_in_title(row['title']) or row['ram'], axis=1)
df_nl['ram'] = df_nl['ram'].fillna(df_nl['RAM'])
df_nl['storage'] = df_nl['storage'].fillna(df_nl['Storage'])
df_nl['cpu'] = df_nl['cpu'].fillna(df_nl['CPU'])
df_nl=df_nl.drop(['Storage','CPU','RAM'], axis=1)
df_nl['storage'] = df_nl['storage'].apply(convert_to_gb)
df_nl['ram'] = df_nl['ram'].apply(convert_to_gb)
df_nl['screen size'] = df_nl['screen size'].apply(convert_screen_size)
df_nl[['width resolution', 'height resolution']] = df_nl['resolution'].apply(convert_resolution).apply(pd.Series)
df_nl['price'] = df_nl['price'].apply(convert_price_before).apply(lambda x: change_price_to_USD(x, EUR_USD))
df_nl['price before promotion'] = df_nl['price before promotion'].apply(convert_price_before).apply(lambda x: change_price_to_USD(x, EUR_USD))
df_nl['rate'] = df_nl['rate'].apply(convert_rate)
df_nl['currancy'] = df_nl['currancy'].apply(change_to_USD)
df_nl